In [151]:
import nltk
import re
import string
from pprint import pprint
from contractions import CONTRACTION_MAP
from nltk.corpus import wordnet
from pattern3.en import suggest
'''Note: need to catch error in empty try, line 37 of 
/Users/minhnguyen/.local/lib/python3.5/site-packages/pattern3/text/tree.py
by replacing try-except with simply assignment izip=zip'''
from nltk.stem import PorterStemmer

In [6]:
paragraph = """The brown fox wasn't that quick and he couldn't win the race. \
Hey that's a great deal! I just bought a phone for $199. \
@@You'll (learn) a **lot** in the book. Python is an amazing language!@@"""

In [144]:
def remove_characters_before_tokenization(sentence, keep_apostrophes=False):
    '''Remove special characters from the sentence'''
    sentence = sentence.strip()
    if keep_apostrophes:
        PATTERN = r'[$|&|*|%|@|(|)|~]' # add other characters here to remove them
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    else:
        PATTERN = r'[^a-zA-Z0-9 ]' # only extract alpha-numeric characters
        filtered_sentence = re.sub(PATTERN, r'', sentence)
    return filtered_sentence

def remove_characters_after_tokenization(tokens):
    pattern = re.compile('[{}]'.format(re.escape(string.punctuation)))
    filtered_tokens = list(filter(None, [pattern.sub('', token) for token in tokens]))
    return filtered_tokens

def expand_contractions(sentence, contraction_mapping):
    '''Expand short form to full words ie. can't => cannot, isn't => is not'''
    contractions_pattern = re.compile('({})'.format('|'.join(contraction_mapping.keys())),
                                      flags=re.IGNORECASE|re.DOTALL)
    def expand_match(contraction):
        match = contraction.group(0)
        first_char = match[0]
        expanded_contraction = contraction_mapping.get(match)\
                                if contraction_mapping.get(match)\
                                else contraction_mapping.get(match.lower())
        expanded_contraction = first_char+expanded_contraction[1:]
        return expanded_contraction
    expanded_sentence = contractions_pattern.sub(expand_match, sentence)
    return expanded_sentence

def remove_stopwords(tokens):
    '''Remove not meaningful words, ie. the, is, a, an.
    BE CAREFUL: this nltk.corpus.stopwords contains even meaningful words ie. not
    so for now, donot touch it.'''
    stopword_list = nltk.corpus.stopwords.words('english')
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
    return filtered_tokens

In [148]:
def tokenize_text(text):
    nochar_text = remove_characters_before_tokenization(text, keep_apostrophes=True)
    text = expand_contractions(nochar_text, CONTRACTION_MAP)
    sentences = nltk.sent_tokenize(text)
    word_tokens = [nltk.word_tokenize(sentence) for sentence in sentences]
    #word_tokens = [remove_stopwords(tokens) for tokens in word_tokens]
    word_tokens = [remove_characters_after_tokenization(tokens) for tokens in word_tokens]
    return word_tokens

In [149]:
def suggest_spelling(tokens):
    repeat_pattern = re.compile(r'(\w*)(\w)\2(\w*)')
    match_substitution = r'\1\2\3'
    def remove_repeated(old_word):
        if wordnet.synsets(old_word):
            return old_word
        new_word = repeat_pattern.sub(match_substitution, old_word)
        return remove_repeated(new_word) if new_word != old_word else new_word
    
    def suggest_spell(old_word):
        # if old_word makes sense, quit
        if wordnet.synsets(old_word):
            return old_word
        
        new_word = old_word
        sugg = suggest(old_word)[0]
        if sugg[1] > 0.4:  # catch any good suggestion
            new_word = sugg[0]
        else:  # if no good suggestion, try removing repeated chars and suggest again
            sugg_1 = suggest(remove_repeated(old_word))[0]
            if sugg_1[1] > 0.4:
                new_word = sugg_1[0]
        
        if wordnet.synsets(new_word):
            return new_word
        return old_word
    
    correct_tokens = [suggest_spell(word) for word in tokens]
    return correct_tokens
#suggest_spelling(['catt', 'finalllyy', 'carrs', 'carries'])

#def lemmatize to get meaningful team

In [150]:
tokenize_text(paragraph)

[['The',
  'brown',
  'fox',
  'was',
  'not',
  'that',
  'quick',
  'and',
  'he',
  'could',
  'not',
  'win',
  'the',
  'race'],
 ['Hey', 'that', 'is', 'a', 'great', 'deal'],
 ['I', 'just', 'bought', 'a', 'phone', 'for', '199'],
 ['You', 'will', 'learn', 'a', 'lot', 'in', 'the', 'book'],
 ['Python', 'is', 'an', 'amazing', 'language']]